# Explaining text classifications 

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

2022-06-02 14:01:14.101618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 14:01:14.101666: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Import Industry Classification Mode
This finetuned model by @sampathkethineedi uses a distilbert base to predict the professional industry a text is referring to. 

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained("sampathkethineedi/industry-classification")

Let's explore the classes, there are 62 unique classes. Many of these are overlapping/related industries such as __Health Care Equipment__ and __Health Care Supplies__.

In [3]:
model.config.id2label

{0: 'Advertising',
 1: 'Aerospace & Defense',
 2: 'Apparel Retail',
 3: 'Apparel, Accessories & Luxury Goods',
 4: 'Application Software',
 5: 'Asset Management & Custody Banks',
 6: 'Auto Parts & Equipment',
 7: 'Biotechnology',
 8: 'Building Products',
 9: 'Casinos & Gaming',
 10: 'Commodity Chemicals',
 11: 'Communications Equipment',
 12: 'Construction & Engineering',
 13: 'Construction Machinery & Heavy Trucks',
 14: 'Consumer Finance',
 15: 'Data Processing & Outsourced Services',
 16: 'Diversified Metals & Mining',
 17: 'Diversified Support Services',
 18: 'Electric Utilities',
 19: 'Electrical Components & Equipment',
 20: 'Electronic Equipment & Instruments',
 21: 'Environmental & Facilities Services',
 22: 'Gold',
 23: 'Health Care Equipment',
 24: 'Health Care Facilities',
 25: 'Health Care Services',
 26: 'Health Care Supplies',
 27: 'Health Care Technology',
 28: 'Homebuilding',
 29: 'Hotels, Resorts & Cruise Lines',
 30: 'Human Resource & Employment Services',
 31: 'IT Co

Import __SequenceClassificationExplainer__ from transformers interpret. This class should work with most if not all language models with a sequence classification head from the transformers package. 


In [4]:
from transformers_interpret import SequenceClassificationExplainer

In [5]:
sample_text = """
Stocks ended a choppy session mixed as investors digested a host of corporate earnings results and considered policymakers’ next moves to support the still virus-stricken economy.
The S&P 500 shook off earlier declines to narrowly eke out a record closing high.The Dow ended a tick below its recent record closing level."""

In [6]:
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

In [7]:
# call the exlplainer
word_attributions = multiclass_explainer(text=sample_text)

In [8]:
# seems to be an appropriate prediction
multiclass_explainer.predicted_class_name

'Investment Banking & Brokerage'

In [9]:
# look the the raw word attributions
word_attributions

[('[CLS]', 0.0),
 ('stocks', 0.17552751301305788),
 ('ended', -0.03850302476937364),
 ('a', 0.00521391054284526),
 ('chop', -0.37112104366102805),
 ('##py', 0.05474087780425294),
 ('session', 0.11222961855917385),
 ('mixed', 0.04523500466782869),
 ('as', 0.008543666523580103),
 ('investors', 0.2255227737782134),
 ('digest', 0.09217342236118296),
 ('##ed', 0.1464443192019554),
 ('a', -0.10249606762941596),
 ('host', -0.07218372671727502),
 ('of', 0.22421000740747335),
 ('corporate', 0.27895953392914824),
 ('earnings', 0.6027407614739859),
 ('results', -0.09964955064564822),
 ('and', -0.017121871003684386),
 ('considered', 0.24777281000809753),
 ('policy', 0.1037567496070395),
 ('##makers', 0.011601935679800668),
 ('’', 0.011302660858373047),
 ('next', -0.04636319579006631),
 ('moves', -0.004164065972683219),
 ('to', 0.08991349800073678),
 ('support', 0.07168823531926437),
 ('the', 0.02782972325264251),
 ('still', 0.07106041187027728),
 ('virus', 0.10655639122413249),
 ('-', -0.073002972

## Visualizating Explanations 
With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to this prediction. **Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

In [10]:
html = multiclass_explainer.visualize()

## Explaining The Same Text For A Different Class 
Lets say we think this text could also fall somewhat under the class of __Asset Management & Custody Banks__ If we want it is also possible to get an explantion/attributions for the text with that class

In [11]:
word_attributions = multiclass_explainer(sample_text, class_name="Asset Management & Custody Banks")

In [12]:
# look the the raw word attributions
word_attributions

[('[CLS]', 0.0),
 ('stocks', -0.14385773469394603),
 ('ended', 0.1711180105590572),
 ('a', -0.060598525287441515),
 ('chop', 0.5140056676968525),
 ('##py', -0.08750486200085784),
 ('session', -0.07754123984446873),
 ('mixed', 0.036185132152124884),
 ('as', -0.00942673176531002),
 ('investors', 0.4988447733610029),
 ('digest', -0.21302120013323178),
 ('##ed', -0.0642910821888625),
 ('a', 0.17772634017094088),
 ('host', 0.15808854470653136),
 ('of', -0.06705071617375406),
 ('corporate', -0.15002057419985015),
 ('earnings', -0.23341458571656326),
 ('results', 0.12625864074402468),
 ('and', 0.057195821055909916),
 ('considered', -0.11990891933703209),
 ('policy', 0.14461759064370744),
 ('##makers', 0.15514268225746658),
 ('’', -0.027600643078955968),
 ('next', 0.02122589524962554),
 ('moves', 0.10101868712245538),
 ('to', -0.052635201975218156),
 ('support', -0.0014980131380415984),
 ('the', 0.08523793721220806),
 ('still', 0.003591248306267035),
 ('virus', 0.008249719637054719),
 ('-', -0

The results are close to the first visualization, a good sign that the model is generalizing well for both of these related classes

In [13]:
html = multiclass_explainer.visualize()

What if we get attributions for class that makes no sense in this context such as __Restaurants__?


In [14]:
word_attributions = multiclass_explainer(sample_text, class_name="Restaurants")

There isn't much to this prediction, it is worth nothing however that the words "choppy" had a more positive impact in this instance which seems plausible given the industry. 

In [15]:
html = multiclass_explainer.visualize()

In [16]:
word_attributions

[('[CLS]', 0.0),
 ('stocks', -0.6854856883169093),
 ('ended', -0.029031849270978665),
 ('a', 0.11090281386769735),
 ('chop', 0.006652195211802581),
 ('##py', -0.20624895616153002),
 ('session', -0.0706553060609612),
 ('mixed', -0.13288198171191423),
 ('as', 0.07492894428515953),
 ('investors', -0.1211925915659857),
 ('digest', -0.12445959138828452),
 ('##ed', -0.09281780986424851),
 ('a', 0.04926237871597633),
 ('host', 0.0024579161027010786),
 ('of', -0.041810129557414935),
 ('corporate', 0.09596871604322571),
 ('earnings', 0.19780773002574376),
 ('results', 0.26498240758279035),
 ('and', 0.10765588459976716),
 ('considered', 0.0880260443510709),
 ('policy', 0.11527471579892593),
 ('##makers', -0.09797678481597628),
 ('’', -0.04952405450571627),
 ('next', -0.08798395803020652),
 ('moves', 0.016330803984313857),
 ('to', -0.01398491295448637),
 ('support', -0.07466646936005066),
 ('the', 0.018439105897551707),
 ('still', -0.07228693526941468),
 ('virus', 0.07444839624063074),
 ('-', -0.